In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when
from datetime import datetime
import os

In [3]:
spark = SparkSession.builder.config("spark.driver.memory", "8g").config("spark.executor.cores", 8).getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/25 11:05:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<h1>Method 2</h1>

In [16]:
def etl_process(path, file_name):
    df = spark.read.json(path + file_name)
    df = df.select('_source.*')

    df = df.withColumn("Type",
                       when((col("AppName") == 'CHANNEL') | (col("AppName") == 'DSHD') | (col("AppName") == 'KPLUS') | (col("AppName") == 'KPlus'), "Truyền Hình")
                       .when((col("AppName") == 'VOD') | (col("AppName") == 'FIMS_RES') | (col("AppName") == 'BHD_RES') |
                             (col("AppName") == 'VOD_RES') | (col("AppName") == 'FIMS') | (col("AppName") == 'BHD') | (
                                     col("AppName") == 'DANET'), "Phim Truyện")
                       .when((col("AppName") == 'RELAX'), "Giải Trí")
                       .when((col("AppName") == 'CHILD'), "Thiếu Nhi")
                       .when((col("AppName") == 'SPORT'), "Thể Thao")
                       .otherwise("Error"))

    df = df.select('Contract', 'Type', 'TotalDuration')
    df = df.filter(df.Type != 'Error')

    result = df.groupBy('Contract', 'Type').sum()
    result = result.withColumnRenamed('sum(TotalDuration)', 'TotalDuration')
    result = df.groupBy("Contract").pivot("Type").sum("TotalDuration")

    print('Finished Processing {}'.format(file_name))
    return result

In [17]:
def main_task(start_date_str, end_date_str):
    path = "/Users/habaokhanh/Study_BigData_Dataset/log_content/"
    list_file = [file for file in os.listdir(path) if file != '.DS_Store']
    
    result = None
    for file_name in list_file:
        date_str = file_name.split('_')[-1].split('.')[0]
        file_date = datetime.strptime(date_str, "%Y%m%d").date()
        
        start_date = datetime.strptime(start_date_str, "%Y%m%d").date()
        end_date = datetime.strptime(end_date_str, "%Y%m%d").date()

        if start_date <= file_date <= end_date:
            df = etl_process(path, file_name)
            if result is None:
                result = df
            else:
                result = result.union(df)

    result = result.groupby('Contract').sum()
    result = result.withColumnRenamed('sum(Giải Trí)', 'RelaxDuration') \
        .withColumnRenamed('sum(Phim Truyện)', 'MovieDuration') \
        .withColumnRenamed('sum(Thiếu Nhi)', 'ChildDuration') \
        .withColumnRenamed('sum(Thể Thao)', 'SportDuration') \
        .withColumnRenamed('sum(Truyền Hình)', 'TVDuration')

    print('-----------Saving Data ---------')
    result.repartition(1).write.csv('/Users/habaokhanh/Study_BigData_Dataset/log_content/clean/df_clean2', header=True)
    
    return print('Data Saved Successfully')


main_task('20220401', '20220402')

Finished Processing 20220402.json
Finished Processing 20220401.json
-----------Saving Data ---------


Data Saved Successfully


In [18]:
df_test=spark.read.csv('/Users/habaokhanh/Study_BigData_Dataset/log_content/clean/df_clean2', header=True)
df_test.show()

+---------+-------------+-------------+-------------+-------------+----------+
| Contract|RelaxDuration|MovieDuration|ChildDuration|SportDuration|TVDuration|
+---------+-------------+-------------+-------------+-------------+----------+
|HNH644322|         null|         null|         null|         null|     19692|
|BED008119|         null|         null|         null|         null|      7220|
|BNFD36683|         null|         null|          315|         null|     15038|
|CBFD04764|         null|         null|         null|         null|    167551|
|GLD024214|         null|          165|         null|         null|     35938|
|HNH795510|         null|        18713|          265|         null|    100481|
|NBFD10014|         null|         null|         null|         null|    115857|
|LDD028485|         null|         null|         null|         null|       328|
|DNFD35509|         null|          872|         null|         null|     85096|
|HND603711|         null|         7319|         null

<h1>Method 1</h1>

In [29]:
def etl_process(path, file_name):
    spark = SparkSession.builder.config("spark.driver.memory", "8g").config("spark.executor.cores",8).getOrCreate()
    
    df = spark.read.json(path+file_name)
    df = df.select('_source.*')
    
    df = df.withColumn("Type",
           when((col("AppName") == 'CHANNEL') | (col("AppName") =='DSHD')| (col("AppName") =='KPLUS')| (col("AppName") =='KPlus'), "Truyền Hình")
        .when((col("AppName") == 'VOD') | (col("AppName") =='FIMS_RES')| (col("AppName") =='BHD_RES')| 
              (col("AppName") =='VOD_RES')| (col("AppName") =='FIMS')| (col("AppName") =='BHD')| (col("AppName") =='DANET'), "Phim Truyện")
        .when((col("AppName") == 'RELAX'), "Giải Trí")
        .when((col("AppName") == 'CHILD'), "Thiếu Nhi")
        .when((col("AppName") == 'SPORT'), "Thể Thao")
        .otherwise("Error"))
    
    df = df.select('Contract','Type','TotalDuration')
    df = df.filter(df.Type != 'Error')
    
    print('Finished Processing {}'.format(file_name))
    return df

In [30]:
def main_task(start_date_str, end_date_str):
    start_time = datetime.now()
    path = '/Users/habaokhanh/Study_BigData_Dataset/log_content/'
    list_file = [file for file in os.listdir(path) if file != '.DS_Store']
            
    result = None
    for file_name in list_file:
        date_str = file_name.split('_')[-1].split('.')[0]
        file_date = datetime.strptime(date_str, "%Y%m%d").date()
        
        start_date = datetime.strptime(start_date_str, "%Y%m%d").date()
        end_date = datetime.strptime(end_date_str, "%Y%m%d").date()

        if start_date <= file_date <= end_date:
            df = etl_process(path, file_name)
            if result is None:
                result = df
            else:
                result = result.union(df)
    
    result = result.groupby('Contract','Type').sum()
    result = result.withColumnRenamed('sum(TotalDuration)','TotalDuration')
    final = result.groupBy("Contract").pivot("Type").sum("TotalDuration")

    final = final.withColumnRenamed('Giải Trí', 'RelaxDuration') \
        .withColumnRenamed('Phim Truyện', 'MovieDuration') \
        .withColumnRenamed('Thiếu Nhi', 'ChildDuration') \
        .withColumnRenamed('Thể Thao', 'SportDuration') \
        .withColumnRenamed('Truyền Hình', 'TVDuration')
    
    print('-----------Saving Data ---------')
    final.repartition(1).write.csv('/Users/habaokhanh/Study_BigData_Dataset/log_content/clean/df_clean1',header=True)
    
    end_time = datetime.now()
    time_processing = (end_time - start_time).total_seconds()
    print('It took {} to process the data'.format(time_processing))
    
    return print('Data Saved Successfully')

main_task('20220401', '20220402')


Finished Processing 20220402.json
Finished Processing 20220401.json


-----------Saving Data ---------


It took 7.16612 to process the data
Data Saved Successfully


In [31]:
df_test=spark.read.csv('/Users/habaokhanh/Study_BigData_Dataset/log_content/clean/df_clean1', header=True)
df_test.show()

+---------+-------------+-------------+-------------+-------------+----------+
| Contract|RelaxDuration|MovieDuration|ChildDuration|SportDuration|TVDuration|
+---------+-------------+-------------+-------------+-------------+----------+
|HNH644322|         null|         null|         null|         null|     19692|
|BED008119|         null|         null|         null|         null|      7220|
|BNFD36683|         null|         null|          315|         null|     15038|
|CBFD04764|         null|         null|         null|         null|    167551|
|GLD024214|         null|          165|         null|         null|     35938|
|HNH795510|         null|        18713|          265|         null|    100481|
|NBFD10014|         null|         null|         null|         null|    115857|
|LDD028485|         null|         null|         null|         null|       328|
|DNFD35509|         null|          872|         null|         null|     85096|
|HND603711|         null|         7319|         null